In [15]:
import os
import pandas as pd

def AADT_splits(df_in):
    # apply ODOT methodology AADT_spit calculations to dataframe and create new columns.
    # df_in['aadt_singl] = Dir_AADT_BUD, 
    # df_in['aadt_combi'] = DirAADT_TRK
    df_in['DirAADT_AUTO'] = df_in['aadt'] - df_in['aadt_singl'] + df_in['aadt_combi']
    df_in['pct_auto'] = df_in['DirAADT_AUTO']/df_in['aadt']
    df_in['pct_bus'] = df_in['aadt_singl']/df_in['aadt']
    df_in['pct_truck'] = df_in['aadt_combi']/df_in['aadt']
    return df_in

def main():
    df = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/Feb2017_test.csv')) #fix in script implementation
    df_meta = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/TMC_Identification.csv'), 
                          usecols=['tmc', 'miles', 'tmclinear', 'aadt', 'aadt_singl', 'aadt_combi' ])
    df_odot = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/odot_edt.csv'))
    df = pd.merge(df, df_meta, left_on=df['tmc_code'], right_on=['tmc'], how='inner')
    df = pd.merge(df, df_odot, left_on=df['tmc_code'], right_on=['TMC'], how='inner')
    df = AADT_splits(df)
    # Return a selected row for test purposes only
    print(df.loc[df['tmc_code'] == '114-04368'])

In [16]:
main()

         tmc_code   measurement_tstamp  speed  average_speed  reference_speed  \
806400  114-04368  2017-02-01 00:00:00  66.00           13.0             62.0   
806401  114-04368  2017-02-01 00:15:00    NaN           13.0             62.0   
806402  114-04368  2017-02-01 00:30:00  55.50           13.0             62.0   
806403  114-04368  2017-02-01 00:45:00  51.26           13.0             62.0   
806404  114-04368  2017-02-01 01:00:00  54.93           18.0             62.0   
806405  114-04368  2017-02-01 01:15:00  53.00           18.0             62.0   
806406  114-04368  2017-02-01 01:30:00    NaN           18.0             62.0   
806407  114-04368  2017-02-01 01:45:00  59.00           18.0             62.0   
806408  114-04368  2017-02-01 02:00:00  63.00           26.0             62.0   
806409  114-04368  2017-02-01 02:15:00  19.35           26.0             62.0   
806410  114-04368  2017-02-01 02:30:00  27.00           26.0             62.0   
806411  114-04368  2017-02-0